In [1]:
#Based off of TM-Evaluate-Models-Worksheet in that instead of getting a score for a test set we are printing bets for 
#a test set

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import RadiusNeighborsClassifier

remove_fight_island = False    #Removes fight island fights from consideration

In [3]:
import sys
sys.path.append('../automated_model_creation') #We need to access the function file

In [4]:
import pandas as pd
import numpy as np
from functions import custom_cv_eval, get_ev_from_df, get_bet_ev, get_bet_return
import random
import csv


In [5]:
df = pd.read_csv("../data/kaggle_data/ufc-master.csv")
df_upcoming = pd.read_csv('../data/kaggle_data/upcoming-event.csv')
len(df)





4813

In [6]:
#The fight island fights were messing with the established models.  This function most likely isn't needed anymore
if remove_fight_island:
    ##Let's remove the Fight island contests and see how that affects score
    df_no_fight_island = df[(df['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    #df_train_no_fight_island = df_train[(df_train['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df = df_no_fight_island
    #df_train = df_train_no_fight_island

len(df)

4813

In [7]:
num_upcoming_fights = len(df_upcoming)

In [8]:
df = df_upcoming.append(df)
len(df)

4826

In [9]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
#df = df[(df['Winner'] != 'Blue') | (df['Winner'] == 'Red') ]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

<ipython-input-9-e6abcb5e0e70>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 0
<ipython-input-9-e6abcb5e0e70>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 1


In [10]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [11]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [12]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[num_upcoming_fights:]
odds_train = odds_df[num_upcoming_fights:]
label_train = label_df[num_upcoming_fights:]

df_test = df[:num_upcoming_fights]
odds_test = odds_df[:num_upcoming_fights]
label_test = label_df[:num_upcoming_fights]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

13
13
13
4813
4813
4813


In [13]:
weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [14]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,label
0,Uriah Hall,Sean Strickland,175,-210,175.000000,47.619048,2021-07-31,"Las Vegas, Nevada, USA",USA,Blue,...,5.0,5:00,1500.0,650.0,225.0,2500.0,800.0,275.0,165.0,1.0
1,Cheyanne Buys,Gloria de Paula,-145,125,68.965517,125.000000,2021-07-31,"Las Vegas, Nevada, USA",USA,Red,...,1.0,1:00,60.0,100.0,200.0,800.0,1400.0,900.0,900.0,0.0
2,Niklas Stolze,Jared Gooden,-180,155,55.555556,155.000000,2021-07-31,"Las Vegas, Nevada, USA",USA,Blue,...,1.0,1:08,68.0,180.0,300.0,600.0,1200.0,300.0,600.0,1.0
3,Collin Anglin,Melsik Baghdasaryan,135,-155,135.000000,64.516129,2021-07-31,"Las Vegas, Nevada, USA",USA,Blue,...,2.0,1:50,410.0,240.0,250.0,1000.0,850.0,650.0,225.0,1.0
4,Bryan Barberena,Jason Witt,-265,215,37.735849,215.000000,2021-07-31,"Las Vegas, Nevada, USA",USA,Blue,...,3.0,5:00,900.0,300.0,500.0,550.0,750.0,120.0,850.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4808,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,1.0,0:44,44.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4809,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,2:01,121.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4810,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,0:47,47.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4811,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,3.0,5:00,900.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,label
0,Derrick Lewis,Ciryl Gane,270,-350,270.000000,28.571429,2021-08-07,"Houston, Texas, USA",USA,NaN,...,NaN,NaN,NaN,1400.0,200.0,2500.0,650.0,400.0,130.0,NaN
1,Jose Aldo,Pedro Munhoz,-120,100,83.333333,100.000000,2021-08-07,"Houston, Texas, USA",USA,NaN,...,NaN,NaN,NaN,150.0,225.0,1600.0,900.0,450.0,450.0,NaN
2,Michael Chiesa,Vicente Luque,-105,-115,95.238095,86.956522,2021-08-07,"Houston, Texas, USA",USA,NaN,...,NaN,NaN,NaN,225.0,225.0,400.0,700.0,1000.0,350.0,NaN
3,Tecia Torres,Angela Hill,-145,125,68.965517,125.000000,2021-08-07,"Houston, Texas, USA",USA,NaN,...,NaN,NaN,NaN,-105.0,180.0,1200.0,2000.0,1400.0,600.0,NaN
4,Song Yadong,Casey Kenney,105,-125,105.000000,80.000000,2021-08-07,"Houston, Texas, USA",USA,NaN,...,NaN,NaN,NaN,215.0,150.0,1200.0,800.0,400.0,700.0,NaN
5,Bobby Green,Rafael Fiziev,250,-320,250.000000,31.250000,2021-08-07,"Houston, Texas, USA",USA,NaN,...,NaN,NaN,NaN,400.0,120.0,2000.0,1400.0,1000.0,175.0,NaN
6,Vince Morales,Drako Rodriguez,-105,-115,95.238095,86.956522,2021-08-07,"Houston, Texas, USA",USA,NaN,...,NaN,NaN,NaN,250.0,225.0,1000.0,450.0,350.0,550.0,NaN
7,Alonzo Menifield,Ed Herman,-240,195,41.666667,195.000000,2021-08-07,"Houston, Texas, USA",USA,NaN,...,NaN,NaN,NaN,500.0,500.0,1200.0,800.0,-135.0,700.0,NaN
8,Karolina Kowalkiewicz,Jessica Penne,-140,120,71.428571,120.000000,2021-08-07,"Houston, Texas, USA",USA,NaN,...,NaN,NaN,NaN,-110.0,200.0,2000.0,600.0,1000.0,1600.0,NaN
9,Manel Kape,Ode Osbourne,-200,170,50.000000,170.000000,2021-08-07,"Houston, Texas, USA",USA,NaN,...,NaN,NaN,NaN,180.0,380.0,1200.0,1100.0,180.0,550.0,NaN


In [15]:
def display_bets(ev_df, min_ev):
    
    for index, row in ev_df.iterrows():
        t1_bet_ev = get_bet_ev(row['t1_odds'], row['t1_prob'])
        t2_bet_ev = get_bet_ev(row['t2_odds'], row['t2_prob'])
        #print(f"ODDS:{row['t2_odds']} PROB: {row['t2_prob']} EV: {t2_bet_ev}")
        t1_bet_return = get_bet_return(row['t1_odds'])
        t2_bet_return = get_bet_return(row['t2_odds'])
        print(f"{row['t1_name']} vs. {row['t2_name']}")
        if t1_bet_ev > min_ev:
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        elif t2_bet_ev > min_ev:
            print(f"{row['t2_name']} has an EV of {round(t2_bet_ev, 2)} on odds of {round(row['t2_odds'],2)}.  They have {round(row['t2_prob']*100,2)}", 
                  "% chance of winning.")
        
        else:
            print(f"No bets.  EV1:{t1_bet_ev}. EV2: {t2_bet_ev}")
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        print()
        
    return(0)
        

In [16]:
def get_bets(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
             test_odds, verbose=True):
    fighters_test = df[['R_fighter', 'B_fighter']]
    #Train model.  
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    #LOOK AT get_ev and prepare the labels and odds
    
    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    fighters_test = fighters_test[fighters_test.index.isin(df_test.index)]
    #print(df_train)
    input_model.fit(df_train, labels_train)
    
    probs = input_model.predict_proba(df_test)
    odds_test = np.array(odds_test)  
    fighters_test = np.array(fighters_test)
    #display(fighters_test)
    prepped_test = list(zip(fighters_test[:, -2], fighters_test[:, -1], odds_test[:, -2], odds_test[:, -1], 
                            probs[:, 0], probs[:, 1], labels_test))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_name', 't2_name', 't1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
    
    #display(ev_prepped_df)
    
    display_bets(ev_prepped_df, input_ev)
    

    return(ev_prepped_df)

In [17]:
with open('../data/production_models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
#print(len(models))

#set the model choice automatically:
model_choice = 0
best_score = models[4][0]
for n in range(len(models[0])):
    print(n)
    if (float(models[4][n]) > float((best_score))):
        print("YES")
        best_score = models[4][n]
        model_choice = n

print(f"The model choice is {model_choice}")
print(models)



model = eval(models[1][model_choice])
features = eval(models[2][model_choice])
ev = eval(models[3][model_choice])
print()
print(model, features, ev)

0
1
2
3
4
5
6
7
8
9
YES
10
11
12
The model choice is 9
[['model_06_13_20', 'model_06_20_20', 'model_6_27_20', 'model_7_18_20', 'model_7_25_20', 'model_9_2_20', 'model_9_10_20', 'model_10_23_20', 'model_10_27_20', 'model_10_28_20', 'model_2021_06_24', 'model_2021_06_30', 'model_2021_07_21'], ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)', 'GaussianNB()', 'GaussianNB()', 'GradientBoostingClassifier(n_estimators=122, random_state=75)', 'GaussianNB()', 'GaussianNB(var_smoothing=1e-12)', 'GaussianNB(var_smoothing=1e-12)', 'GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,\n                           random_state=75)', "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,\n                       n_jobs=-1, random_state=75)", "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=22,\n                       n_jobs=-1, random_state=75)", 'GradientBoostingClassifier(max_depth=95, min_samples_leaf

In [18]:

output_df = get_bets(model, features, ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)


Derrick Lewis vs. Ciryl Gane
Derrick Lewis has an EV of 87.86 on odds of 270.  They have 50.77 % chance of winning.

Jose Aldo vs. Pedro Munhoz
Pedro Munhoz has an EV of 34.59 on odds of 100.  They have 67.29 % chance of winning.

Michael Chiesa vs. Vicente Luque
No bets.  EV1:-3.0825217240840317. EV2: -5.849928826248274
Michael Chiesa has an EV of -3.08 on odds of -105.  They have 49.64 % chance of winning.

Tecia Torres vs. Angela Hill
Angela Hill has an EV of 28.42 on odds of 125.  They have 57.08 % chance of winning.

Song Yadong vs. Casey Kenney
Song Yadong has an EV of 22.59 on odds of 105.  They have 59.8 % chance of winning.

Bobby Green vs. Rafael Fiziev
Bobby Green has an EV of 38.62 on odds of 250.  They have 39.61 % chance of winning.

Vince Morales vs. Drako Rodriguez
No bets.  EV1:-2.2068591134732216. EV2: -6.688447741817285
Vince Morales has an EV of -2.21 on odds of -105.  They have 50.09 % chance of winning.

Alonzo Menifield vs. Ed Herman
No bets.  EV1:-7.261771625633

In [19]:
output_df

,t1_name,t2_name,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,Derrick Lewis,Ciryl Gane,270,-350,0.507729,0.492271,NaN
1,Jose Aldo,Pedro Munhoz,-120,100,0.327062,0.672938,NaN
2,Michael Chiesa,Vicente Luque,-105,-115,0.496407,0.503593,NaN
3,Tecia Torres,Angela Hill,-145,125,0.429241,0.570759,NaN
4,Song Yadong,Casey Kenney,105,-125,0.598024,0.401976,NaN
5,Bobby Green,Rafael Fiziev,250,-320,0.396062,0.603938,NaN
6,Vince Morales,Drako Rodriguez,-105,-115,0.500892,0.499108,NaN
7,Alonzo Menifield,Ed Herman,-240,195,0.654623,0.345377,NaN
8,Karolina Kowalkiewicz,Jessica Penne,-140,120,0.513713,0.486287,NaN
9,Manel Kape,Ode Osbourne,-200,170,0.694706,0.305294,NaN


In [20]:
return_df = output_df[['t1_name', 't2_name', 't1_prob', 't2_prob']]

In [21]:
return_df

,t1_name,t2_name,t1_prob,t2_prob
0,Derrick Lewis,Ciryl Gane,0.507729,0.492271
1,Jose Aldo,Pedro Munhoz,0.327062,0.672938
2,Michael Chiesa,Vicente Luque,0.496407,0.503593
3,Tecia Torres,Angela Hill,0.429241,0.570759
4,Song Yadong,Casey Kenney,0.598024,0.401976
5,Bobby Green,Rafael Fiziev,0.396062,0.603938
6,Vince Morales,Drako Rodriguez,0.500892,0.499108
7,Alonzo Menifield,Ed Herman,0.654623,0.345377
8,Karolina Kowalkiewicz,Jessica Penne,0.513713,0.486287
9,Manel Kape,Ode Osbourne,0.694706,0.305294


In [22]:
output_df = return_df.rename(columns = {'t1_name': 'R_fighter', 't2_name': 'B_fighter', 't1_prob': 'R_prob', 't2_prob': 'B_prob'})

In [23]:
output_df.to_csv('../data/upcoming_probs.csv', index=False)